In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torch import optim
from tqdm.auto import tqdm
import math
torch.manual_seed(55)

# read in data

In [2]:
path = "W://Study Material/Jupyter Notebook/Datasets/CIFAR-10/cifar-10-batches-py"
def unpickle(file):
    import pickle
    with open(file, "rb") as fo:
        dict = pickle.load(fo,encoding="bytes")
    return dict

In [3]:
training_batch = np.empty((4),dtype=object)
val_batch = np.empty((1),dtype=object)
test_batch = np.empty((1),dtype=object)
training_batch.shape

(4,)

In [4]:
training_batch[0] = unpickle(path+"/data_batch_1")
training_batch[1] = unpickle(path+"/data_batch_2")
training_batch[2] = unpickle(path+"/data_batch_3")
training_batch[3] = unpickle(path+"/data_batch_4")
val_batch[0] = unpickle(path+"/data_batch_5")
test_batch[0] = unpickle(path+"/test_batch")
meta_data = unpickle(path+"/batches.meta")

In [5]:
training_batch[0].keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [6]:
np.array(training_batch[0][b'labels']).shape

(10000,)

In [7]:
training_batch[0][b'data'].shape

(10000, 3072)

In [8]:
meta_data[b'label_names']

[b'airplane',
 b'automobile',
 b'bird',
 b'cat',
 b'deer',
 b'dog',
 b'frog',
 b'horse',
 b'ship',
 b'truck']

# define custom data loader

In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,batch,b_type,transform):
        self.transform = transform
        if b_type == "train":
            self.data = np.concatenate((batch[0][b'data'],batch[1][b'data'],
                                        batch[2][b'data'],batch[3][b'data']),axis=0)
            #print(b_type, " data shape: ",self.data.shape)
            self.label = np.concatenate((batch[0][b'labels'],batch[1][b'labels'],
                                        batch[2][b'labels'],batch[3][b'labels']))
            #print(b_type, " label shape: ",len(self.label))
        else:
            self.data = batch[0][b'data']
            #print(b_type, " data shape: ",self.data.shape)
            self.label = batch[0][b'labels']
            #print(b_type, " label shape: ", len(self.label))
    
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self,index):
        label = self.label[index]
        image_data = self.data[index]
        # convert (3072,) array to (3,32,32)
        image_r = image_data[:1024].reshape(32,32)
        image_g = image_data[1024:2048].reshape(32,32)
        image_b = image_data[2048:].reshape(32,32)
        image = np.array([image_r,image_g,image_b])
        image = np.transpose(image,(1,2,0)) # change the batch dimension to the last dimension
        #image = torch.tensor(image,dtype=torch.float)
        if self.transform is not None:
            image = self.transform(image)
        #print(image.shape)
        return image, label
    

In [10]:
x = np.ones((1,2,3))
np.transpose(x,(2,0,1)).shape

(3, 1, 2)

### transformation

In [11]:
# define data augmentation
data_transformers = {"train": transforms.Compose([transforms.ToPILImage(),
                                     transforms.RandomResizedCrop(256),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.RandomRotation(30),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]),
            "test":transforms.Compose([transforms.ToPILImage(),
                                       transforms.Resize(256),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])}

### datasets

In [12]:
# defining datasets
train_data = CustomDataset(training_batch,"train",transform=data_transformers["train"])
val_data = CustomDataset(val_batch,"validation",transform=data_transformers["test"])
test_data = CustomDataset(test_batch,"test",transform=data_transformers["test"])

In [13]:
# make sure the shape of the image extracted is okay
print(train_data[0][0].shape)
#print(train_data[0][1])

torch.Size([3, 256, 256])


### dataloader

In [14]:
train_dataloader = DataLoader(train_data,batch_size=128,shuffle=True)
val_dataloader = DataLoader(val_data,batch_size=128,shuffle=True)
test_dataloader = DataLoader(test_data,batch_size=128,shuffle=False)

# training and validation routine

In [20]:
def train(model,trainloader,valloader,epochs,print_frequency,loss_fn,optimizer,device,run,part):
    training_steps = epochs * (len(trainloader))
    progress_bar = tqdm(range(training_steps))
    ep = epochs
    print_every = math.floor(training_steps/print_frequency)
    steps = 0
    model.to(device)
    model.train()
    total = 0
    correct = 0
    best_val = 10000
    
    current_loss = 0
    
    for e in np.arange(ep):
        for batch,(images,labels) in enumerate(trainloader):
            steps += 1
            images, labels = images.to(device), labels.to(device)
            # convert images to float because weights are floats
            images = images.float()
            labels = labels.type(torch.long) # need to be int
            
            # calculate loss and backpropogate
            optimizer.zero_grad()
            outputs = model(images)
            _,predictions = torch.max(outputs.data,1)
            total += labels.size(0)
            correct += (predictions == labels).sum().item()
            loss = loss_fn(outputs,labels)
            loss.backward()
            optimizer.step()
            
            current_loss += loss.item()
            progress_bar.update(1)
            
            if steps % print_every == 0:
                print('EPOCHS : {}/{}'.format(e+1,epochs),
                      'Loss : {:.6f}'.format(current_loss/print_every)) 
                print('The training accuracy is {:.2f}%'.format(correct/total*100))
                current_loss = 0
                val_loss = validate(model,valloader,loss_fn, device)
                if val_loss < best_val:
                    torch.save(model.state_dict(),"Weights/part-{}-run-{}-Best.pth".format(str(part),
                                                                                           str(run)))
    torch.save(model.state_dict(),"Weights/part-{}-run-{}-Last.pth".format(str(part),
                                                                           str(run)))

In [21]:
def validate(model, valloader, loss_fn, device):
    total = 0
    correct = 0
    val_loss = 0
    model.eval()
    steps = 0
    with torch.no_grad():
        for batch, (images,labels) in enumerate(valloader):
            images, labels = images.to(device), labels.to(device)
            # convert images to float because weights are floats
            images = images.float()
            labels = labels.type(torch.long) # need to be int
            steps += 1
            outputs = model(images)
            _, predictions = torch.max(outputs.data,1)
            total += labels.size(0)
            correct += (labels == predictions).sum().item()
            loss = loss_fn(outputs,labels)
            val_loss += loss.item()
    val_loss /= steps
    accuracy = correct / total * 100
    print("The validation loss is %.4f" % (val_loss))
    print('The valudation accuracy is {:.2f}%\n'.format(accuracy))
    return val_loss

# Part b)

In [17]:
AlexNet = models.alexnet(pretrained=False)
AlexNet.load_state_dict(torch.load("W://Study Material/Jupyter Notebook/Pretrained_Weights/alexnet-owt-7be5be79.pth"))
AlexNet.cuda()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(AlexNet.parameters(),lr=1e-3)

In [24]:
train(AlexNet,train_dataloader,val_dataloader,10,20,loss_fn,optimizer,"cuda",1)

  0%|          | 0/3130 [00:00<?, ?it/s]

EPOCHS : 1/10 Loss : 2.247014
The training accuracy is 21.80%
The validation loss is 1.6998
The valudation accuracy is 36.71%

EPOCHS : 1/10 Loss : 1.785275
The training accuracy is 27.76%
The validation loss is 1.5200
The valudation accuracy is 43.46%

EPOCHS : 2/10 Loss : 1.685747
The training accuracy is 31.12%
The validation loss is 1.4152
The valudation accuracy is 47.11%

EPOCHS : 2/10 Loss : 1.600968
The training accuracy is 33.61%
The validation loss is 1.3193
The valudation accuracy is 51.00%

EPOCHS : 3/10 Loss : 1.554594
The training accuracy is 35.58%
The validation loss is 1.3243
The valudation accuracy is 53.44%

EPOCHS : 3/10 Loss : 1.490266
The training accuracy is 37.28%
The validation loss is 1.1996
The valudation accuracy is 56.95%

EPOCHS : 4/10 Loss : 1.486563
The training accuracy is 38.56%
The validation loss is 1.1369
The valudation accuracy is 58.76%

EPOCHS : 4/10 Loss : 1.429415
The training accuracy is 39.73%
The validation loss is 1.1196
The valudation accu

In [25]:
optimizer = optim.Adam(AlexNet.parameters(),lr=1e-5)
train(AlexNet,train_dataloader,val_dataloader,10,20,loss_fn,optimizer,"cuda",2)

  0%|          | 0/3130 [00:00<?, ?it/s]

EPOCHS : 1/10 Loss : 1.314707
The training accuracy is 54.08%
The validation loss is 0.7874
The valudation accuracy is 72.85%

EPOCHS : 1/10 Loss : 1.110474
The training accuracy is 57.41%
The validation loss is 0.7677
The valudation accuracy is 73.35%

EPOCHS : 2/10 Loss : 1.099722
The training accuracy is 58.54%
The validation loss is 0.7579
The valudation accuracy is 73.52%

EPOCHS : 2/10 Loss : 1.087660
The training accuracy is 59.29%
The validation loss is 0.7448
The valudation accuracy is 74.05%

EPOCHS : 3/10 Loss : 1.072100
The training accuracy is 59.91%
The validation loss is 0.7409
The valudation accuracy is 74.36%

EPOCHS : 3/10 Loss : 1.068306
The training accuracy is 60.23%
The validation loss is 0.7364
The valudation accuracy is 74.53%

EPOCHS : 4/10 Loss : 1.055259
The training accuracy is 60.57%
The validation loss is 0.7403
The valudation accuracy is 74.46%

EPOCHS : 4/10 Loss : 1.051216
The training accuracy is 60.83%
The validation loss is 0.7277
The valudation accu

In [19]:
optimizer = optim.Adam(AlexNet.parameters(),lr=1e-6)
AlexNet.load_state_dict(torch.load("Weights/run-2-Best.pth"))
train(AlexNet,train_dataloader,val_dataloader,10,20,loss_fn,optimizer,"cuda",3)

  0%|          | 0/3130 [00:00<?, ?it/s]

EPOCHS : 1/10 Loss : 1.236205
The training accuracy is 57.47%
The validation loss is 0.7003
The valudation accuracy is 75.54%

EPOCHS : 1/10 Loss : 1.020405
The training accuracy is 60.70%
The validation loss is 0.7034
The valudation accuracy is 75.55%

EPOCHS : 2/10 Loss : 1.018962
The training accuracy is 61.69%
The validation loss is 0.6994
The valudation accuracy is 75.49%

EPOCHS : 2/10 Loss : 1.014674
The training accuracy is 62.28%
The validation loss is 0.6986
The valudation accuracy is 75.52%

EPOCHS : 3/10 Loss : 1.023155
The training accuracy is 62.52%
The validation loss is 0.7007
The valudation accuracy is 75.56%

EPOCHS : 3/10 Loss : 1.013412
The training accuracy is 62.75%
The validation loss is 0.7012
The valudation accuracy is 75.43%

EPOCHS : 4/10 Loss : 1.010199
The training accuracy is 62.95%
The validation loss is 0.7021
The valudation accuracy is 75.45%

EPOCHS : 4/10 Loss : 1.028888
The training accuracy is 63.05%
The validation loss is 0.7022
The valudation accu

# Part c)

In [26]:
AlexNet2 = models.alexnet(pretrained=False)
AlexNet2.load_state_dict(torch.load("W://Study Material/Jupyter Notebook/Pretrained_Weights/alexnet-owt-7be5be79.pth"))
AlexNet2.cuda()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [27]:
AlexNet2.classifier = nn.Sequential(
    nn.Dropout(0.5,False),
    nn.Linear(9216,4096,True),
    nn.ReLU(True),
    nn.Linear(4096,10,True)
)
print(AlexNet2)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [24]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(AlexNet2.parameters(),lr=1e-3)

In [25]:
train(AlexNet2,train_dataloader,val_dataloader,10,20,loss_fn,optimizer,"cuda",1,"c")

  0%|          | 0/3130 [00:00<?, ?it/s]

EPOCHS : 1/10 Loss : 2.342232
The training accuracy is 9.85%
The validation loss is 2.3030
The valudation accuracy is 9.52%

EPOCHS : 1/10 Loss : 2.302850
The training accuracy is 9.81%
The validation loss is 2.3027
The valudation accuracy is 10.14%

EPOCHS : 2/10 Loss : 2.302646
The training accuracy is 9.81%
The validation loss is 2.3028
The valudation accuracy is 10.14%

EPOCHS : 2/10 Loss : 2.302732
The training accuracy is 9.78%
The validation loss is 2.3027
The valudation accuracy is 10.03%



KeyboardInterrupt: 

In [28]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(AlexNet2.parameters(),lr=1e-5)

In [29]:
train(AlexNet2,train_dataloader,val_dataloader,10,20,loss_fn,optimizer,"cuda",1,"c")

  0%|          | 0/3130 [00:00<?, ?it/s]

EPOCHS : 1/10 Loss : 1.749224
The training accuracy is 37.26%
The validation loss is 1.0611
The valudation accuracy is 62.49%

EPOCHS : 1/10 Loss : 1.344720
The training accuracy is 44.61%
The validation loss is 0.8322
The valudation accuracy is 71.62%

EPOCHS : 2/10 Loss : 1.248277
The training accuracy is 48.05%
The validation loss is 0.7669
The valudation accuracy is 73.68%

EPOCHS : 2/10 Loss : 1.182601
The training accuracy is 50.66%
The validation loss is 0.7867
The valudation accuracy is 72.40%

EPOCHS : 3/10 Loss : 1.149564
The training accuracy is 52.32%
The validation loss is 0.6671
The valudation accuracy is 76.95%

EPOCHS : 3/10 Loss : 1.132360
The training accuracy is 53.53%
The validation loss is 0.6508
The valudation accuracy is 77.58%

EPOCHS : 4/10 Loss : 1.089950
The training accuracy is 54.61%
The validation loss is 0.6348
The valudation accuracy is 78.25%

EPOCHS : 4/10 Loss : 1.067065
The training accuracy is 55.53%
The validation loss is 0.5904
The valudation accu

In [30]:
optimizer = optim.Adam(AlexNet2.parameters(),lr=1e-7)
train(AlexNet2,train_dataloader,val_dataloader,10,20,loss_fn,optimizer,"cuda",2,"c")

  0%|          | 0/3130 [00:00<?, ?it/s]

EPOCHS : 1/10 Loss : 0.983893
The training accuracy is 65.21%
The validation loss is 0.4936
The valudation accuracy is 83.12%

EPOCHS : 1/10 Loss : 0.904120
The training accuracy is 66.81%
The validation loss is 0.4828
The valudation accuracy is 83.32%

EPOCHS : 2/10 Loss : 0.906161
The training accuracy is 67.25%
The validation loss is 0.4769
The valudation accuracy is 83.34%

EPOCHS : 2/10 Loss : 0.900338
The training accuracy is 67.54%
The validation loss is 0.4763
The valudation accuracy is 83.48%

EPOCHS : 3/10 Loss : 0.921114
The training accuracy is 67.56%
The validation loss is 0.4737
The valudation accuracy is 83.46%

EPOCHS : 3/10 Loss : 0.895418
The training accuracy is 67.72%
The validation loss is 0.4758
The valudation accuracy is 83.38%

EPOCHS : 4/10 Loss : 0.905156
The training accuracy is 67.83%
The validation loss is 0.4768
The valudation accuracy is 83.53%

EPOCHS : 4/10 Loss : 0.887147
The training accuracy is 67.99%
The validation loss is 0.4795
The valudation accu

In [31]:
optimizer = optim.Adam(AlexNet2.parameters(),lr=1e-8)
#AlexNet.load_state_dict(torch.load("Weights/part-c-run-2-Best.pth"))
train(AlexNet2,train_dataloader,val_dataloader,10,20,loss_fn,optimizer,"cuda",3,"c")

  0%|          | 0/3130 [00:00<?, ?it/s]

EPOCHS : 1/10 Loss : 0.960729
The training accuracy is 65.96%
The validation loss is 0.4728
The valudation accuracy is 83.51%

EPOCHS : 1/10 Loss : 0.884539
The training accuracy is 67.56%
The validation loss is 0.4713
The valudation accuracy is 83.52%

EPOCHS : 2/10 Loss : 0.894978
The training accuracy is 67.94%
The validation loss is 0.4709
The valudation accuracy is 83.56%

EPOCHS : 2/10 Loss : 0.900989
The training accuracy is 68.03%
The validation loss is 0.4811
The valudation accuracy is 83.58%

EPOCHS : 3/10 Loss : 0.895619
The training accuracy is 68.20%
The validation loss is 0.4710
The valudation accuracy is 83.62%

EPOCHS : 3/10 Loss : 0.904552
The training accuracy is 68.23%
The validation loss is 0.4682
The valudation accuracy is 83.59%

EPOCHS : 4/10 Loss : 0.898023
The training accuracy is 68.27%
The validation loss is 0.4718
The valudation accuracy is 83.64%

EPOCHS : 4/10 Loss : 0.896598
The training accuracy is 68.29%
The validation loss is 0.4702
The valudation accu

# Part d)

In [32]:
AlexNet3 = models.alexnet(pretrained=False)
AlexNet3.load_state_dict(torch.load("W://Study Material/Jupyter Notebook/Pretrained_Weights/alexnet-owt-7be5be79.pth"))
AlexNet3.cuda()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [34]:
AlexNet3.classifier[6] = nn.Linear(4096,10,True)
print(AlexNet3)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [35]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(AlexNet3.parameters(),lr=1e-5)

In [36]:
train(AlexNet3,train_dataloader,val_dataloader,10,20,loss_fn,optimizer,"cuda",1,"d")

  0%|          | 0/3130 [00:00<?, ?it/s]

EPOCHS : 1/10 Loss : 1.850561
The training accuracy is 32.47%
The validation loss is 1.2623
The valudation accuracy is 55.76%

EPOCHS : 1/10 Loss : 1.405315
The training accuracy is 40.75%
The validation loss is 0.9747
The valudation accuracy is 65.92%

EPOCHS : 2/10 Loss : 1.257076
The training accuracy is 45.55%
The validation loss is 0.8586
The valudation accuracy is 69.45%

EPOCHS : 2/10 Loss : 1.177570
The training accuracy is 48.66%
The validation loss is 0.7658
The valudation accuracy is 73.17%

EPOCHS : 3/10 Loss : 1.127938
The training accuracy is 50.88%
The validation loss is 0.6838
The valudation accuracy is 76.32%

EPOCHS : 3/10 Loss : 1.089887
The training accuracy is 52.55%
The validation loss is 0.6361
The valudation accuracy is 78.05%

EPOCHS : 4/10 Loss : 1.061739
The training accuracy is 53.94%
The validation loss is 0.6482
The valudation accuracy is 77.48%

EPOCHS : 4/10 Loss : 1.048316
The training accuracy is 55.06%
The validation loss is 0.5840
The valudation accu

In [37]:
optimizer = optim.Adam(AlexNet3.parameters(),lr=1e-6)
train(AlexNet3,train_dataloader,val_dataloader,10,20,loss_fn,optimizer,"cuda",2,"d")

  0%|          | 0/3130 [00:00<?, ?it/s]

EPOCHS : 1/10 Loss : 1.000398
The training accuracy is 64.65%
The validation loss is 0.4515
The valudation accuracy is 84.18%

EPOCHS : 1/10 Loss : 0.840051
The training accuracy is 67.60%
The validation loss is 0.4455
The valudation accuracy is 84.52%

EPOCHS : 2/10 Loss : 0.841163
The training accuracy is 68.52%
The validation loss is 0.4427
The valudation accuracy is 84.39%

EPOCHS : 2/10 Loss : 0.834193
The training accuracy is 69.07%
The validation loss is 0.4453
The valudation accuracy is 84.51%

EPOCHS : 3/10 Loss : 0.842139
The training accuracy is 69.33%
The validation loss is 0.4452
The valudation accuracy is 84.52%

EPOCHS : 3/10 Loss : 0.832247
The training accuracy is 69.59%
The validation loss is 0.4508
The valudation accuracy is 84.45%

EPOCHS : 4/10 Loss : 0.829938
The training accuracy is 69.76%
The validation loss is 0.4495
The valudation accuracy is 84.32%

EPOCHS : 4/10 Loss : 0.840508
The training accuracy is 69.87%
The validation loss is 0.4373
The valudation accu

In [38]:
optimizer = optim.Adam(AlexNet3.parameters(),lr=1e-8)
#AlexNet.load_state_dict(torch.load("Weights/part-d-run-2-Best.pth"))
train(AlexNet3,train_dataloader,val_dataloader,10,20,loss_fn,optimizer,"cuda",3,"d")

  0%|          | 0/3130 [00:00<?, ?it/s]

EPOCHS : 1/10 Loss : 0.991773
The training accuracy is 64.99%
The validation loss is 0.4360
The valudation accuracy is 84.82%

EPOCHS : 1/10 Loss : 0.805074
The training accuracy is 68.39%
The validation loss is 0.4337
The valudation accuracy is 84.83%

EPOCHS : 2/10 Loss : 0.811057
The training accuracy is 69.44%
The validation loss is 0.4410
The valudation accuracy is 84.85%

EPOCHS : 2/10 Loss : 0.822433
The training accuracy is 69.90%
The validation loss is 0.4325
The valudation accuracy is 84.87%

EPOCHS : 3/10 Loss : 0.820671
The training accuracy is 70.17%
The validation loss is 0.4350
The valudation accuracy is 84.86%

EPOCHS : 3/10 Loss : 0.810344
The training accuracy is 70.41%
The validation loss is 0.4347
The valudation accuracy is 84.86%

EPOCHS : 4/10 Loss : 0.813434
The training accuracy is 70.58%
The validation loss is 0.4321
The valudation accuracy is 84.92%

EPOCHS : 4/10 Loss : 0.816192
The training accuracy is 70.63%
The validation loss is 0.4329
The valudation accu